# CCM Logical workflow and testing journal

This notebook will be used for processing and refining the CCM script.
All algorithms used in this workflow are derived from FM 5-33

In [118]:
from IPython.display import Image

In [ ]:
# Import Statements
# Script function parameters ===============================
import os, sys, math, traceback, random, types
import arcpy
from arcpy import env
from arcpy import da
from arcpy import sa


# Environments Settings

# ==================================================

In [ ]:
arcpy.env.workspace = r"E:\02_Work\02_GIS\01_Topo_WS\03_Analysis\Special\03_Test_Plans\AnalysisOverlays.gdb"
workspace = arcpy.env.workspace
arcpy.env.overwriteOutput = True


# Defining Constants

First, the dictionaries used in this script will need to be created. 
This includes the following dictionaries:
vehicleDict 
slopeCat

#  ==================================================

### TODO: Determine how to calculate/resolve None type fields

In [24]:
# if errors occur in computation, assign all values as float by adding .0 to each value
vehicleDict = {'M1':{'maxRoadSpeed':71,'maxOnRoadGrad':68.7,'maxOffRoadGrad':53, 'vehicleWidth':3.65, 'overrideDiameterBreast':0.25, 'VCI1':25, 'VCI50':58,'minTurnRad':9.9,'vehLength':9.9, 'milLoadClass':60},
              'M60A1':{'maxRoadSpeed':48,'maxOnRoadGrad':60,'maxOffRoadGrad':45,'vehicleWidth':3.63, 'overrideDiameterBreast':0.15, 'VCI1':20, 'VCI50':48,'minTurnRad':9.4,'vehLength':9.4, 'milLoadClass':54},
               'M109':{'maxRoadSpeed':56,'maxOnRoadGrad':60,'maxOffRoadGrad':45,'vehicleWidth':3.10, 'overrideDiameterBreast':0.12, 'VCI1':25, 'VCI50':57,'minTurnRad':6.6,'vehLength':6.6, 'milLoadClass':24},
              'M113':{'maxRoadSpeed':48,'maxOnRoadGrad':60,'maxOffRoadGrad':45,'vehicleWidth':2.69, 'overrideDiameterBreast':0.10, 'VCI1':17, 'VCI50':40,'minTurnRad':4.8,'vehLength':4.8, 'milLoadClass':12},
              'M35A2':{'maxRoadSpeed':56,'maxOnRoadGrad':64,'maxOffRoadGrad':30,'vehicleWidth':2.43, 'overrideDiameterBreast':0.06, 'VCI1':26, 'VCI50':59,'minTurnRad':5.3,'vehLength':6.7, 'milLoadClass':10},
              'M151':{'maxRoadSpeed':50,'maxOnRoadGrad':60,'maxOffRoadGrad':28,'vehicleWidth':1.69, 'overrideDiameterBreast':0.04, 'VCI1':19, 'VCI50':44,'minTurnRad':5.8,'vehLength':3.35, 'milLoadClass':None},
              'T62':{'maxRoadSpeed':50,'maxOnRoadGrad':62,'maxOffRoadGrad':45,'vehicleWidth':3.37, 'overrideDiameterBreast':0.15, 'VCI1':21, 'VCI50':49,'minTurnRad':9.33,'vehLength':9.33, 'milLoadClass':42},
              'T72':{'maxRoadSpeed':60,'maxOnRoadGrad':62,'maxOffRoadGrad':45,'vehicleWidth':3.38, 'overrideDiameterBreast':0.18, 'VCI1':25, 'VCI50':60,'minTurnRad':9.2,'vehLength':9.2, 'milLoadClass':45},
              'M2':{'maxRoadSpeed':66,'maxOnRoadGrad':60,'maxOffRoadGrad':None,'vehicleWidth':6.2, 'overrideDiameterBreast':None, 'VCI1':15, 'VCI50':35,'minTurnRad':6.2,'vehLength':6.2, 'milLoadClass':24},
              'M3':{'maxRoadSpeed':66,'maxOnRoadGrad':60,'maxOffRoadGrad':None,'vehicleWidth':6.2, 'overrideDiameterBreast':None, 'VCI1':15, 'VCI50':35,'minTurnRad':6.2,'vehLength':6.2, 'milLoadClass':24},
              'M48AVLB':{'maxRoadSpeed':48,'maxOnRoadGrad':30,'maxOffRoadGrad':None,'vehicleWidth':4.3, 'overrideDiameterBreast':None, 'VCI1':26, 'VCI50':60,'minTurnRad':9.6,'vehLength':9.6, 'milLoadClass':62},
              'M60AVLB':{'maxRoadSpeed':51,'maxOnRoadGrad':30,'maxOffRoadGrad':None,'vehicleWidth':4.3, 'overrideDiameterBreast':None, 'VCI1':22, 'VCI50':51,'minTurnRad':9.6,'vehLength':9.6, 'milLoadClass':63},
              'MLR5':{'maxRoadSpeed':64,'maxOnRoadGrad':60,'maxOffRoadGrad':None,'vehicleWidth':2.97, 'overrideDiameterBreast':None, 'VCI1':None, 'VCI50':None,'minTurnRad':6.97,'vehLength':6.97, 'milLoadClass':28}}
vehicleDict

{'M1': {'VCI1': 25,
  'VCI50': 58,
  'maxOffRoadGrad': 53,
  'maxOnRoadGrad': 68.7,
  'maxRoadSpeed': 71,
  'milLoadClass': 60,
  'minTurnRad': 9.9,
  'overrideDiameterBreast': 0.25,
  'vehLength': 9.9,
  'vehicleWidth': 3.65},
 'M109': {'VCI1': 25,
  'VCI50': 57,
  'maxOffRoadGrad': 45,
  'maxOnRoadGrad': 60,
  'maxRoadSpeed': 56,
  'milLoadClass': 24,
  'minTurnRad': 6.6,
  'overrideDiameterBreast': 0.12,
  'vehLength': 6.6,
  'vehicleWidth': 3.1},
 'M113': {'VCI1': 17,
  'VCI50': 40,
  'maxOffRoadGrad': 45,
  'maxOnRoadGrad': 60,
  'maxRoadSpeed': 48,
  'milLoadClass': 12,
  'minTurnRad': 4.8,
  'overrideDiameterBreast': 0.1,
  'vehLength': 4.8,
  'vehicleWidth': 2.69},
 'M151': {'VCI1': 19,
  'VCI50': 44,
  'maxOffRoadGrad': 28,
  'maxOnRoadGrad': 60,
  'maxRoadSpeed': 50,
  'milLoadClass': None,
  'minTurnRad': 5.8,
  'overrideDiameterBreast': 0.04,
  'vehLength': 3.35,
  'vehicleWidth': 1.69},
 'M2': {'VCI1': 15,
  'VCI50': 35,
  'maxOffRoadGrad': None,
  'maxOnRoadGrad': 60,
  '

In [20]:
slopeCategory = {1: 1.5, 2: 6.5, 3:15.0, 4:25.0, 5:35.0, 6:37.5,7:0}
slopeCategory

gslope = int(input())
slpval = slopeCategory[gslope]
print(slpval)

5
35.0


In [6]:
print(slopeCategory[1])

1.5


## Dictionaries defined
## Arguments for F1--- ==> Speed/Slope Category

In [25]:
# queary statement for specs from M1

print('Please choose from the list below to determine vehicle specs:\n', vehicleDict.keys(), '\n')
print('Inputs are case sensitive')
## variable assigned for vehicle of choice in list
ccmVehicle = str(input())# test 'M1' = 71

## variable for calling sub dictionary vehicle type (i.e.., M1, M60A1, M109 etc...)
dict_of_info = vehicleDict[ccmVehicle]

## variable storing the results of the hehicle types specific sub attribute
query1 = dict_of_info['maxOffRoadGrad']
print('The Max Off Road Gradability is: ', query1, '%')

Please choose from the list below to determine vehicle specs:
 dict_keys(['M1', 'M60A1', 'M109', 'M113', 'M35A2', 'M151', 'T62', 'T72', 'M2', 'M3', 'M48AVLB', 'M60AVLB', 'MLR5']) 

Inputs are case sensitive
M1
The Max Off Road Gradability is:  53 %


In [ ]:
maxOffGrad = dict_of_info['maxOffRoadGrad'] # Test for val == 53, 
maxOffGrad
if maxOffGrad == 53:
    print(True)

In [ ]:
# Formula check to verify F1 = 24
F1 = (45-15)/(60/48)  # Answer is float((45.0 - 15.0)/(60.0 / 48.0))
F1

# Working solution for None type values in condition

In [ ]:
## formula check using variables for calc

##assign variables

maxOffGrad = dict_of_info['maxOffRoadGrad']
gSlope = 15
maxOnGrad = dict_of_info['maxOnRoadGrad']
maxSpeed = dict_of_info['maxRoadSpeed']
print(maxOffGrad,gSlope,maxOnGrad,maxSpeed)

## why does this equal zero???
## pulls random values from dict, need callers for each sub-query
for val in dict_of_info:
    if val == None:
        None == 0
        F1 = float((maxOffGrad-gSlope)/(maxOnGrad/maxSpeed))
        print(maxOffGrad)
        if F1 > 0:
            F1 = 0

print("This does not validate against M1, recheck full script", F1)
#print(float((0-15)/(60/64)))

In [ ]:
##assign variables
print('Please choose from the list below to determine vehicle specs:\n', vehicleDict.keys(), '\n')
print('Inputs are case sensitive')
## variable assigned for vehicle of choice in list
ccmVehicle = str(input())# test 'M1' = 71

## variable for calling sub dictionary vehicle type (i.e.., M1, M60A1, M109 etc...)
dict_of_info = vehicleDict[ccmVehicle]

## variable storing the results of the hehicle types specific sub attribute
print('The expression used to evaluate for ground/slope(F1) for ', ccmVehicle, 'is:')
maxOffGrad = dict_of_info['maxOffRoadGrad']
gSlope = 15
maxOnGrad = dict_of_info['maxOnRoadGrad']
maxSpeed = dict_of_info['maxRoadSpeed']
# need if statement to handle None values(assume most restrictive... None == NoGO or Zero)

# Argument for calculating F1 rounded to nearest hundredth
print('(',maxOffGrad,'-',gSlope,')','/','(',maxOnGrad,'/',maxSpeed,')')

## pulls random values from dict, need callers for each sub-query
F1 = round((maxOffGrad-gSlope)/(maxOnGrad/maxSpeed),2)
print("\n    The speed slope factor determines the extent that any slope \n  will deteriorate the speed of the vehicle without consideration\n  for any other physical factor.\n")
print("The speed/slope factor represented by F1 is: ", F1)

# Working Arguments for selecting GSC values

In [23]:
# Variables for query of Slope Category
print( slopeCategory.keys())
print('Please choose from the list to determine slope category:')

## variable assigned for GSC grid code in list
gslope = int(input())# test '5' = 35

## variable for calling slope item from gslope input
slpval = slopeCategory[gslope] # ex. gslope (5) = slpval(35)
print('The Grid Code value for ', gslope, 'is ', slpval)

dict_keys([1, 2, 3, 4, 5, 6, 7])
Please choose from the list to determine slope category:
5
The Grid Code value for  5 is  35.0


# Argument for F1 using Dictionaries

This section should finalize the arguments from the 2 dictionaries and call in all the 
values stored in the listed variables then return a value based on the expression and the formula for F1

In [26]:
## variable storing the results of the slope values
print('The expression used to evaluate for ground/slope(F1) for ', ccmVehicle, 'is:')
maxOffGrad = dict_of_info['maxOffRoadGrad']
maxOnGrad = dict_of_info['maxOnRoadGrad']
maxSpeed = dict_of_info['maxRoadSpeed']
# need if statement to handle None values(assume most restrictive... None == NoGO or Zero)

# Argument for calculating F1 rounded to nearest hundredth
print('(',maxOffGrad,'-',slpval,')','/','(',maxOnGrad,'/',maxSpeed,')')

## pulls random values from dict, need callers for each sub-query
F1 = round((maxOffGrad-slpval)/(maxOnGrad/maxSpeed),2)
print("\n    The speed slope factor determines the extent that any slope \n  will deteriorate the speed of the vehicle without consideration\n  for any other physical factor.\n")
print("The speed/slope factor represented by F1 is: ", F1)

The expression used to evaluate for ground/slope(F1) for  M1 is:
( 53 - 35.0 ) / ( 68.7 / 71 )

    The speed slope factor determines the extent that any slope 
  will deteriorate the speed of the vehicle without consideration
  for any other physical factor.

The speed/slope factor represented by F1 is:  18.6


In [ ]:
## example of rounding function used to control outputs and ensure they are rounded to the nearest hundredth
print( round(F1,2))

# Final F1 (Excludes Heading/Constants/Environments)

In [30]:
##assign variables
print('Please choose from the list below to determine vehicle specs:\n', vehicleDict.keys(), '\n')
print('Inputs are case sensitive')
## variable assigned for vehicle of choice in list
ccmVehicle = str(input())# test 'M1' = 71

## variable for calling sub dictionary vehicle type (i.e.., M1, M60A1, M109 etc...)
dict_of_info = vehicleDict[ccmVehicle]

# Variables for query of Slope Category
print('Please choose from the below list to determine slope category:')
print('\n',slopeCategory.keys())

## variable assigned for GSC grid code in list
gslope = int(input())# test '5' = 35

## variable for calling slope item from gslope input
slpval = slopeCategory[gslope] # ex. gslope (5) = slpval(35)
print('The Grid Code value for ', gslope, 'is ', slpval)

## variable storing the results of the slope values
print('The expression used to evaluate for ground/slope(F1) for ', ccmVehicle, 'is:')
maxOffGrad = dict_of_info['maxOffRoadGrad']
maxOnGrad = dict_of_info['maxOnRoadGrad']
maxSpeed = dict_of_info['maxRoadSpeed']
# need if statement to handle None values(assume most restrictive... None == NoGO or Zero)

# Argument for calculating F1 rounded to nearest hundredth
print('(',maxOffGrad,'-',slpval,')','/','(',maxOnGrad,'/',maxSpeed,')')

## pulls random values from dict, need callers for each sub-query
F1 = round((maxOffGrad-slpval)/(maxOnGrad/maxSpeed),2)
print("\n    The speed slope factor determines the extent that any slope \n  will deteriorate the speed of the vehicle without consideration\n  for any other physical factor.\n")
print("The speed/slope factor represented by F1 is: ", F1)

Please choose from the list below to determine vehicle specs:
 dict_keys(['M1', 'M60A1', 'M109', 'M113', 'M35A2', 'M151', 'T62', 'T72', 'M2', 'M3', 'M48AVLB', 'M60AVLB', 'MLR5']) 

Inputs are case sensitive
M1
Please choose from the below list to determine slope category:

 dict_keys([1, 2, 3, 4, 5, 6, 7])
3
The Grid Code value for  3 is  15.0
The expression used to evaluate for ground/slope(F1) for  M1 is:
( 53 - 15.0 ) / ( 68.7 / 71 )

    The speed slope factor determines the extent that any slope 
  will deteriorate the speed of the vehicle without consideration
  for any other physical factor.

The speed/slope factor represented by F1 is:  39.27


# Using environments & scratch space

Create new feature class in working directory and print items from existing feature classes from existing datasets

In [31]:
import os, sys, math, traceback, random, types
import arcpy
from arcpy import env
from arcpy import da
from arcpy import sa

arcpy.env.workspace = r"E:\02_Work\02_GIS\01_Topo_WS\03_Analysis\Special\03_Test_Plans\AnalysisOverlays.gdb"
workspace = arcpy.env.workspace
arcpy.env.overwriteOutput = True

In [88]:
# Featureclass input variables

slope = r"E:\02_Work\02_GIS\01_Topo_WS\03_Analysis\Special\03_Test_Plans\MCGDBv2.gdb\Specialized\PhysiographySrfExt"
surfmat = r"E:\02_Work\02_GIS\01_Topo_WS\03_Analysis\Special\03_Test_Plans\MCGDBv2.gdb\Specialized\PhysiographySrf"
hydro = r"E:\02_Work\02_GIS\01_Topo_WS\03_Analysis\Special\03_Test_Plans\MCGDBv2.gdb\Specialized\HydrographySrf"
synthFeature = arcpy.CreateFeatureclass_management(out_path=workspace,out_name='syth2')


Feature created Successfully using arcpy module. The next thing I need to do is to query against the existing feature classes. I want the associated table values from MCGDB to be selected and moved to the synthFeature:
Create table SGCL in SynthFeature
    for SGCL in PhysiographySrf
        if SGCL NOT Null:
            copy data and paste in SGCL in SynthFeature

In [89]:
## Add fields to SynthFeature
## Note* This section is intended to be removed once the CCM is finalized since the synthFeature 
## should be a permanent feature in the future and may not need to be created each time tool is ran
arcpy.AddFields_management(synthFeature,[['SGCL','DOUBLE'],
                                         ['SDCL','DOUBLE'],
                                         ['F1','DOUBLE'],
                                         ['F2','DOUBLE']])


<Result 'E:\\02_Work\\02_GIS\\01_Topo_WS\\03_Analysis\\Special\\03_Test_Plans\\AnalysisOverlays.gdb\\syth2'>

# Cursor didn't work, need better argument

In [68]:
# Create the search and update features respectively

with arcpy.da.UpdateCursor(synthFeature,['SGCL']) as cursor:
    for row in cursor:
        Get values from slope and populate SGCL
        row = Copy_management(slope,'SGCL')
        cursor.updateRow(row)


TypeError: string indices must be integers

# Using Field Mapping for script

## Need ALL fields and object maps before creating final code

#### Field Mapping works to copy selected features into the target synthesis feature

In [91]:
# Create the required FieldMap and FieldMappings objects
fm_sgcl = arcpy.FieldMap()
fm_sdcl = arcpy.FieldMap()

fms = arcpy.FieldMappings()

# Get the field names 
slp_sgcl = "SGCL"
smc_sdcl = "SDCL"

# Add fields to their corresponding FieldMap objects

fm_sgcl.addInputField(slope, slp_sgcl)
fm_sdcl.addInputField(surfmat,smc_sdcl)

# Set the output field properties for both FieldMap objects
sgcl_name = fm_sgcl.outputField
sgcl_name.name = 'SGCL'
fm_sgcl.outputField = sgcl_name

sdcl_name = fm_sdcl.outputField
sdcl_name.name = 'SDCL'
fm_sdcl.outputField = sdcl_name

# Add the FieldMap objects to the FieldMappings object
fms.addFieldMap(fm_sgcl)
fms.addFieldMap(fm_sdcl)

# Merge the two feature classes
arcpy.Merge_management([slope, surfmat], synthFeature, fms)

del fm_sgcl, fm_sdcl

Field mapping is dropping my empty F1 and F2 fields after executing. Try creating F1 after field map.

In [94]:
arcpy.AddField_management(synthFeature,'F1','DOUBLE')

<Result 'E:\\02_Work\\02_GIS\\01_Topo_WS\\03_Analysis\\Special\\03_Test_Plans\\AnalysisOverlays.gdb\\syth2'>

# At this point, I have a Featureclass named synthFeature.
# Synthfeature contains the fields SGCL, SDCL, and F1
# I now need to use the synthfeature to calculate F1

Pre code logic
using the user selected input for vehicle type, iterate over the F1 field by calculating for each item in SGCL table.

Below is execution of F1 in ArcGIS Pro

![image](files/pics/F1_Calc.jpg)

In [119]:
# Assuming Table is built inside function and may be where code is required to exist
arcpy.management.CalculateField("syth2", "F1", "CalcF1(!SGCL!)", "PYTHON3", r"slopeMedians = {1:1.5, 2:6.5, 3:15.0, 4:25.0, 5:35.0, 6:37.5, 7:0}\ndef CalcF1(SGCL):\n    F1 = float((28.0 - slopeMedians[SGCL]) / (60.0/50))\n    return F1\n")
print ("Pass")

KeyError: None

KeyError: None

KeyError: None

KeyError: None

KeyError: None

...

KeyError: None

KeyError: None

KeyError: None

KeyError: None

Pass


# ================================================







#                          Combining script logic with field map to calculate F1                  









# ================================================

For this step, I may need to use the arcpy function CalculateField_management to correctly calculate for F1. Basic argument should be:

for values in synthFeature==> 'F1'
where ccmvehicle and slpval are used in expression
calculate F1
need to create expression for code block

        arcpy.CalculateField_management(synthFeature,"F1",expression,"PYTHON_9.3",block)

maybe store first argument as expression for F1 where F1 is already defined

In [101]:
print("Failed function")

Failed function


In [ ]:
# Try Creating custom function
# Calculating F1 
# create f1 function
f1Expression = "CalcF1(!SGCL!)"
# Do not run code, will crash Jupyter
f1CodeBlock = """
def CalcF1(SGCL):\n    F1 = round((maxOffGrad-slpval[SGCL])/(maxOnGrad/maxSpeed),2)\n    return"""
arcpy.CalculateField_management(synthFeature,'F1',f1Expression,"PYTHON_9.3",f1CodeBlock)

# Arguments for F2--- ==> Slope-intercept-frequency (SIF)

In [ ]:
## assuming adjusted count for main argument
# F2 = 280 - SIF count (adjusted) / 280

# CalcF2 == name of function, SGCL == input field, Assume float input == field value at given input

F2 = (280 - float(input()))/280
print(round(F2,2))


In [ ]:
##example if/elif/else: statement using arcpy function to calculate field values
## argument from arcpy.CalcualateField_management()
def CalcF2(SGCL):
    if SGCL == 1:
        F2 = float(1.0)
        return F2
    elif SGCL == 2:
        F2 = float(0.99)
        return F2
    elif SGCL == 3:
        F2 = float(0.98)
        return F2
    elif SGCL == 4:
        F2 = float(0.97)
        return F2
    elif SGCL == 5:
        F2 = float(0.96)
        return F2
    elif SGCL == 6:
        F2 = float(0.95)
        return F2
    elif SGCL == 7:
        F2 = float(0.96)
        return F2
__esri_field_calculator_splitter__
CalcF2( !SGCL!)

In [ ]:
# verify calling single value
slopeCategory = {'0-3': 1.5, '3-10': 6.5, '10-20':15.0, '20-30':25.0, '30-45':37.5, '>45':0}
print(slopeCategory['0-3'])

In [ ]:
# begin loop by stepping into each possible argument
# F2 = 280 - SIF count (adjusted) / 280
print("Enter a Slope category to calculate the slope intercept frequency")
SGCL = float(input())

# CalcF2 == name of function, SGCL == input field, Assume float input == field value at given input
for SGCL in slopeCategory:
    if SGCL ==1:
        calF2 = float(1.0)
    elif SGCL ==2:
        calF2 = float(.99)
    elif SGCL ==3:
        calF2 = float(.98)
    elif SGCL ==4:
        calF2 = float(.97)
    elif SGCL ==5:
        calF2 = float(.96)
    elif SGCL ==6:
        calF2 = float(.95)
    elif SGCL ==7:
        calF2 = float(.96)
    else:
        calcF2 = None
        
F2 = F2 = (280 - SGCL)/280
print(round(F2,2))